<a href="https://colab.research.google.com/github/OliviaSoaresB/BD2/blob/main/Pr%C3%A1tica_BD2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**PRÁTICA BD2**

In [ ]:
connection = sqlite3.connect("financas.sqlite")

query = """
create table usuario(
	id_usuario integer primary key autoincrement,
  nome varchar(100),
  email varchar(100),
  celular char(15),
  senha varchar(100)
);
"""
execute_query(connection, query)

query = """
create table card(
	id_card integer primary key autoincrement,
  descricao text,
  resumo varchar (140),
  titulo varchar(100)
);
"""
execute_query(connection, query)

query = """
create table avaliacao(
	id_avaliacao integer primary key autoincrement,
  nota float,
  comentario varchar(140),
  dataAvaliacao date
);
"""
execute_query(connection, query)

query = """
create table vocabulario(
	id_vocabulario integer primary key autoincrement,
  tituloVocabulario varchar(140)
);
"""
execute_query(connection, query)

# -- Relacionamento
query = """
create table avaliar (
    id_usuario integer,
    id_avaliacao integer,
    primary key (id_usuario, id_avaliacao),
    foreign key (id_usuario) references Usuario(id_usuario),
    foreign key (id_avaliacao) references Avaliacao(id_avaliacao)
);
"""
execute_query(connection, query)

query = """
create table visualizar (
    id_usuario integer,
    id_vocabulario integer,
    primary key (id_usuario, id_vocabulario),
    foreign key (id_usuario) references Usuario(id_usuario),
    foreign key (id_vocabulario) references Vocabulario(id_vocabulario)
);
"""
execute_query(connection, query)

query = """
create table favoritar (
    id_usuario integer,
    id_card integer,
    primary key (id_usuario, id_card),
    foreign key (id_usuario) references Usuario(id_usuario),
    foreign key (id_card) references Card(id_card)
);
"""
execute_query(connection, query)

Query create table usuario executada.
Query create table card executada.
Query create table avaliacao executada.
Query create table vocabulario executada.
Query create table avaliar executada.
Query create table visualizar executada.
Query create table favoritar executada.


# **Atividade 01**

**Consulta 1**

**Objetivo da consulta:** Identificar quais cards são mais populares, verificando quantos usuários favoritaram cada um. Essa funcionalidade pode ser usada no back-end para montar uma área de “cards em destaque” dentro da aplicação.

A consulta retorna o título de cada card junto com a quantidade de vezes em que ele foi favoritado, em ordem decrescente. O sistema poderá exibir os cards mais relevantes para os usuários, incentivando o engajamento.

In [ ]:
SELECT c.titulo, COUNT(f.id_usuario) AS qtd_favoritos
FROM card c
LEFT JOIN favoritar f ON c.id_card = f.id_card
GROUP BY c.id_card, c.titulo
ORDER BY qtd_favoritos DESC;


**Consulta 2**

**Objetivo da consulta:** Calcular a média das avaliações atribuídas por cada usuário, para entender o comportamento de avaliação dentro do sistema. Essa informação pode ser usada para relatórios ou até para personalizar recomendações de conteúdo.

A consulta retorna o nome de cada usuário junto da média de todas as notas que ele atribuiu. Assim, é possível visualizar quais usuários costumam avaliar melhor ou pior os conteúdos, permitindo análises no back-end sobre engajamento e perfil de avaliador.

In [ ]:
SELECT u.nome, AVG(a.nota) AS media_avaliacoes
FROM usuario u
JOIN avaliar av ON u.id_usuario = av.id_usuario
JOIN avaliacao a ON av.id_avaliacao = a.id_avaliacao
GROUP BY u.id_usuario, u.nome
ORDER BY media_avaliacoes DESC;


# **Atividade 02**

**Consulta com 3 Entidades**

Exibir quais usuários avaliaram conteúdos e, ao mesmo tempo, quais cards eles favoritaram. Essa consulta pode ser usada para gerar relatórios de engajamento e analisar se existe relação entre usuários que avaliam e os que também favoritam cards.

A consulta retorna o nome do usuário, o título do card que ele favoritou, além da nota e do comentário da avaliação que ele fez. Assim, o sistema pode identificar padrões de uso, como usuários mais ativos ou cards mais bem avaliados e ao mesmo tempo favoritados.





In [ ]:
SELECT u.nome, c.titulo, a.nota, a.comentario
FROM usuario u
JOIN avaliar av ON u.id_usuario = av.id_usuario
JOIN avaliacao a ON av.id_avaliacao = a.id_avaliacao
JOIN favoritar f ON u.id_usuario = f.id_usuario
JOIN card c ON f.id_card = c.id_card;

#**Atividade 03**


Na aplicação de PI3, é importante exibir frequentemente os cards mais populares, ou seja, aqueles mais favoritados pelos usuários. Como essa consulta será realizada diversas vezes (por exemplo, para exibir em uma página inicial ou em relatórios de engajamento), faz sentido criar uma view que já traga essa informação pronta, evitando reescrever a mesma query repetidamente no back-end.

Descrição da ação esperada ao rodar a consulta:
Ao consultar a view vw_cards_populares, o sistema retorna uma lista de cards acompanhados da quantidade de vezes em que foram favoritados, em ordem decrescente. Dessa forma, o sistema pode exibir de forma simples os conteúdos mais relevantes sem precisar rodar novamente a query completa.



In [ ]:
CREATE VIEW vw_cards_populares AS
SELECT c.titulo, COUNT(f.id_usuario) AS qtd_favoritos
FROM card c
LEFT JOIN favoritar f ON c.id_card = f.id_card
GROUP BY c.id_card, c.titulo
ORDER BY qtd_favoritos DESC;


#**Atividade 04**


A inserção de avaliações é uma ação recorrente no PI3, já que vários usuários poderão avaliar conteúdos constantemente. Como envolve duas operações encadeadas (inserir na tabela avaliacao e também registrar o relacionamento em avaliar), é melhor encapsular essa lógica em um procedure, garantindo que não haja inconsistência nos dados e simplificando o código do back-end.

Ao chamar a procedure inserir_avaliacao, o sistema insere automaticamente uma nova avaliação com a nota, comentário e data fornecidos, e já cria o vínculo entre o usuário e essa avaliação. Assim, o processo de adicionar avaliações se torna mais rápido, seguro e reutilizável.


In [ ]:
CREATE PROCEDURE inserir_avaliacao (
    IN p_id_usuario INT,
    IN p_nota FLOAT,
    IN p_comentario VARCHAR(140),
    IN p_data DATE
)
BEGIN
    INSERT INTO avaliacao (nota, comentario, dataAvaliacao)
    VALUES (p_nota, p_comentario, p_data);

    INSERT INTO avaliar (id_usuario, id_avaliacao)
    VALUES (p_id_usuario, LAST_INSERT_ID());
END;

#**Atividade 05**


Calcular a média de avaliações de cada usuário é uma operação recorrente na aplicação, por exemplo para gerar relatórios, personalizar recomendações ou exibir o perfil de engajamento do usuário. Encapsular essa lógica em uma function evita reescrever a mesma query várias vezes no back-end e permite que o cálculo seja chamado de forma simples dentro de outras consultas SQL.

A função media_avaliacoes_usuario recebe como parâmetro o id_usuario e retorna a média das notas dadas por esse usuário. Assim, ao executar SELECT media_avaliacoes_usuario(1);, por exemplo, o sistema retorna a média das avaliações feitas pelo usuário de ID 1.

In [ ]:
CREATE FUNCTION media_avaliacoes_usuario (p_id_usuario INT)
RETURNS FLOAT
DETERMINISTIC
BEGIN
    DECLARE v_media FLOAT;

    SELECT AVG(a.nota)
    INTO v_media
    FROM avaliacao a
    JOIN avaliar av ON a.id_avaliacao = av.id_avaliacao
    WHERE av.id_usuario = p_id_usuario;

    RETURN v_media;
END;

#**Atividade 06**

No PI3, os usuários irão registrar avaliações com frequência. Muitas vezes, a data da avaliação pode não ser informada no momento da inserção. Para evitar erros ou registros incompletos, o trigger garante que sempre exista uma data associada à avaliação, mantendo a integridade e consistência dos dados sem depender do back-end para validar isso.

O trigger trg_data_avaliacao é disparado antes de uma inserção na tabela avaliacao. Se a coluna dataAvaliacao não tiver valor informado, o trigger insere automaticamente a data atual. Assim, cada avaliação terá sempre a informação correta de quando foi registrada.


In [ ]:
CREATE TRIGGER trg_data_avaliacao
BEFORE INSERT ON avaliacao
FOR EACH ROW
BEGIN
    IF NEW.dataAvaliacao IS NULL THEN
        SET NEW.dataAvaliacao = CURRENT_DATE();
    END IF;
END;
